<a href="https://colab.research.google.com/github/jogarrid/nuscenes/blob/master/LSTMmodelNuscenes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import numpy as np
map_data_split = np.load('/content/gdrive/My Drive/map_data_split.npy')  #scenes x partitions x instances x time_steps x 2


In [0]:
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
from tensorflow.contrib import rnn

#define constants[0:train_N]
time_steps=4
#hidden LSTM units
num_units= 64*3

#learning rate for adam
learning_rate=0.001

#size of batch
batch_size=128

train_N = 97 #number of scenes used for training
test_N = 100 - train_N

In [94]:
train_data.shape

(5962, 2, 10)

In [0]:
class Data():
   def __init__(self, map_data_split, train = True):
      #Given data for a list of scenes in mapa_data_split, saves the data necessaryfor prediction.    
      threshold = 0.3
      means = []
      stds = []
      data = [] #list of standardized partitions belonging to all scenes. 
      indexes = [] #Save information regarding which scene, partition, and instance a segment of data belongs to 
      
      for scene_ix in range(map_data_split.shape[0]):
        for partition_ix in range(map_data_split[scene_ix].shape[0]):
          for instance_ix in range(map_data_split[scene_ix][partition_ix].shape[0]):
            traj_x = map_data_split[scene_ix, partition_ix][instance_ix, :, 0]
            traj_y = map_data_split[scene_ix, partition_ix][instance_ix, :, 1]

            if(~np.any(traj_x == -999) and ~np.any(traj_y == -999)):
              mean_x = np.mean(traj_x[0:4])
              mean_y = np.mean(traj_y[0:4])
              std_x = np.std(traj_x[0:4])
              std_y = np.std(traj_y[0:4])
              #if(std_x > threshold or std_y>threshold):
              if(std_x > threshold):
                traj_x = (traj_x - mean_x)/std_x
              else:
                traj_x = traj_x -mean_x
    
              if(std_y > threshold):
                traj_y = (traj_y - mean_y)/std_y
              else: 
                 traj_y = traj_y - mean_y
                
              means.append([mean_x, mean_y])
              stds.append([std_x, std_y])
              data.append([traj_x, traj_y])
              indexes.append([scene_ix, partition_ix, instance_ix])


      self.data = np.array(data)
      self.data_copy= list(data)
     
      self.indexes = np.array(indexes)
      self.means = np.array(means)
      self.stds = np.array(stds)

In [0]:
#For now, both train and test sets include all the dataset.
trainSet = Data(map_data_split[0:train_N])
testSet = Data(map_data_split[train_N:], train = False)

In [0]:
class Model():
  def __init__(self, time_steps, units, batch_size, learning_rate):
    super(Model, self).__init__()
    n_output = 2     #this model does regression on one number
    self.num_units = units
    self.time_steps = time_steps
    
    self.batch_size = batch_size
    self.graph = tf.Graph()
    with self.graph.as_default():
      self.out_weights=tf.Variable(tf.random_normal([num_units,n_output]))
      self.out_bias=tf.Variable(tf.random_normal([n_output]))

      #defining placeholders
      #input image placeholder
      self.x=tf.placeholder("float",[None,2]) #batch_size x n_input
      #input label placeholder
      self.y=tf.placeholder("float",[None,n_output])

      #processing the input tensor from [batch_size,n_steps,n_input] to "time_steps" number of [batch_size,n_input] tensors
      #self.input=tf.unstack(self.x ,time_steps, 1)
      
      #defining the network
      self.lstm=rnn.BasicLSTMCell(num_units, state_is_tuple = False)
      
      self.initial_state=tf.placeholder("float",[None, 2*units]) #batch_size x n_hidden. n_hidden = 2 * num_units ¿?
      
      
      self.outputs, self.state = self.lstm(self.x, self.initial_state)
      
      #converting last output of dimension [batch_size,num_units] to [batch_size,n_classes] by out_weight multiplication
      self.prediction=tf.matmul(self.outputs, self.out_weights)+self.out_bias

      #loss_function
      self.loss=tf.reduce_mean(tf.losses.mean_squared_error(self.y, self.prediction))
      
      #optimization
      self.opt=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(self.loss)

      
  def train(self, fx_divided, directory, stds_ ):
    #fx_divided needs to be  an array of shape samples,  2, time_steps
    threshold = 0
    losst = 0
    iterations = 500
    self.directory = directory
    with self.graph.as_default():
      init=tf.global_variables_initializer()
      self.saver = tf.train.Saver()
      with tf.Session() as sess:
        sess.run(init)
        for iter in range(iterations):
          indexesBatch = np.random.randint(0, fx_divided.shape[0], self.batch_size)
          #batch_y = y[indexesBatch]
          batch = np.zeros((self.batch_size, self.time_steps, 2))  #there are 10 time_steps 
          batch_y = np.zeros((self.batch_size, self.time_steps, 2))  #there are 10 time_steps 

          for w in range(self.batch_size):
            batch[w] = fx_divided[indexesBatch[w]].T #samples, time_steps, 2
            batch_y[w] = fx_divided[indexesBatch[w]].T
            
          preds = []
          
          numpy_state = self.lstm.zero_state(batch_size, dtype=tf.float32).eval()          
          for j in range(self.time_steps-1):
            numpy_state, los, pred = sess.run([self.state,  self.loss, self.prediction], 
                                      feed_dict={self.initial_state: numpy_state, self.x: batch[:,j],
                                                 self.y: batch[:,j+1] })
            if(j == 8):
              sess.run(self.opt,feed_dict={self.initial_state: numpy_state, self.x: batch[:,j],
                                                 self.y: batch[:,j+1] })
            preds.append(pred[0][0])
            losst += los
          if iter % 100==0:
              print("For iter ",iter)
              print("Loss average",losst/(100*(time_steps-1)))
              losst = 0

        self.save_path = self.saver.save(sess, '/content/gdrive/My Drive/model/' + self.directory)
        print(self.save_path)
 
          
  def predict(self, inputt, numpy_state):
    #This code is very slow as you have to access two graphs, for x and for y, before predicting each sample, and to access each model you need to read a file from disk
    with self.graph.as_default():   
      with tf.Session() as sess:
        self.saver.restore(sess, self.save_path)
        
        if(numpy_state.size == 0):
          numpy_state = self.lstm.zero_state(1, dtype=tf.float32).eval()
          
        inputt = inputt.reshape(1, 2)
        pred, numpy_state = sess.run([self.prediction, self.state], feed_dict={self.x: inputt, self.initial_state: numpy_state})
 
    return pred[0], numpy_state

In [0]:
#ADD NOISE; MORE ITERATIONS. Both models X and Y should be the same.
#evaluation. Linnear prediction.  COmpare linnear and LSTM


In [59]:
train_data = trainSet.data

stds_ = trainSet.stds

#Create and train models#
time_steps = 10
modelx = Model(time_steps , num_units, batch_size, learning_rate)

modelx.train(train_data, 'modelx1', stds_) #Using samples 0-3, we predict sample 4


For iter  0
Loss average 0.10316101564301385
For iter  100
Loss average 4.617595730887519
For iter  200
Loss average 3.1832373509473273
For iter  300
Loss average 1.8905944274862607
For iter  400
Loss average 2.0444339735971555
/content/gdrive/My Drive/model/modelx1


In [60]:
train_data

array([[[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00]],

       [[-9.25000000e-02, -9.25000000e-02,  3.97500000e-01, ...,
         -1.72500000e-01, -2.02500000e-01, -2.42500000e-01],
        [ 2.25000000e-02,  2.25000000e-02, -7.50000000e-03, ...,
         -1.77500000e-01, -2.07500000e-01, -2.47500000e-01]],

       [[-1.37781661e+00, -3.75144166e-01,  4.15800184e-01, ...,
          4.97138620e+00,  5.84892259e+00,  6.72645898e+00],
        [ 1.25645840e+00,  4.39760441e-01, -2.19880220e-01, ...,
         -6.03100033e+00, -6.87910975e+00, -7.75863063e+00]],

       ...,

       [[ 1.35143729e+00,  4.38851883e-01, -4.60139628e-01, ...,
         -4.99489238e+00, -6.08962463e+00, -7.17404833e+00],
        [ 1.34832235e+00,  4.41551892e-01, -4.56055676e-01, ...,
         -5.02235170e+00, -6.11756

In [61]:
n_predict = 6 #predict 3 seconds (6 samples) based on the previous 4 samples (2 seconds)
L = 50
train_data = trainSet.data
train_data_copy =  trainSet.data_copy
stds = trainSet.stds
means = trainSet.means

data_predicted = np.zeros((L, train_data.shape[1], train_data.shape[2])) #added -1
#Create prediction
inputt = np.zeros((1,2))
with modelx.graph.as_default():   
   with tf.Session() as sess:
      modelx.saver.restore(sess, modelx.save_path)

      for i in range(L):
        numpy_statex = modelx.lstm.zero_state(1, dtype=tf.float32).eval()
        for j in range(time_steps):
          if(j<4):
            inputt[0,:] = np.array(train_data[i, :, j]).T
          else: 
            inputt[0,:] = [predx, predy]
          inputt = inputt.reshape(1, 2)
          pred, numpy_statex = sess.run([modelx.prediction, modelx.state], feed_dict={modelx.x: inputt, modelx.initial_state: numpy_statex})
   
          predx = pred[0][0]
          predy= pred[0][1]
          data_predicted[i,:,j] = inputt[0,:]

        print(data_predicted[i])
        print(train_data[i])

        data_predicted[i,0,:] = data_predicted[i,0,:]*(stds[i,0]) + means[i,0]
        data_predicted[i,1,:] = data_predicted[i,1,:]*(stds[i,1]) + means[i,1]

#add mean 
#Measure mean squared error 

INFO:tensorflow:Restoring parameters from /content/gdrive/My Drive/model/modelx1
[[0.         0.         0.         0.         0.00466007 0.01743549
  0.03911978 0.07203883 0.11963469 0.18677223]
 [0.         0.         0.         0.         0.09584513 0.21051502
  0.3614541  0.54545289 0.75898886 1.00006104]]
[[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  3.63797881e-12 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]
[[-0.0925     -0.0925      0.3975     -0.2125     -0.1448831  -0.20129704
  -0.25813568 -0.31405658 -0.36805016 -0.41832185]
 [ 0.0225      0.0225     -0.0075     -0.0375      0.02753893  0.11462575
   0.24249899  0.41071641  0.61820048  0.86411411]]
[[-0.0925 -0.0925  0.3975 -0.2125 -0.1925 -0.1625 -0.1325 -0.1725 -0.2025
  -0.2425]
 [ 0.0225  0.0225 -0.0075 -0.0375 -0.

In [62]:
train_data[0:20]

array([[[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  3.63797881e-12,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00]],

       [[-9.25000000e-02, -9.25000000e-02,  3.97500000e-01,
         -2.12500000e-01, -1.92500000e-01, -1.62500000e-01,
         -1.32500000e-01, -1.72500000e-01, -2.02500000e-01,
         -2.42500000e-01],
        [ 2.25000000e-02,  2.25000000e-02, -7.50000000e-03,
         -3.75000000e-02, -7.75000000e-02, -1.07500000e-01,
         -1.47500000e-01, -1.77500000e-01, -2.07500000e-01,
         -2.47500000e-01]],

       [[-1.37781661e+00, -3.75144166e-01,  4.15800184e-01,
          1.33716059e+00,  2.22472899e+00,  3.15928940e+00,
          4.09384981e+00,  4.97138620e+00,  5.84

In [55]:
train_data

array([[[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00]],

       [[-9.25000000e-02, -9.25000000e-02,  3.97500000e-01, ...,
         -1.72500000e-01, -2.02500000e-01, -2.42500000e-01],
        [ 2.25000000e-02,  2.25000000e-02, -7.50000000e-03, ...,
         -1.77500000e-01, -2.07500000e-01, -2.47500000e-01]],

       [[-1.37781661e+00, -3.75144166e-01,  4.15800184e-01, ...,
          4.97138620e+00,  5.84892259e+00,  6.72645898e+00],
        [ 1.25645840e+00,  4.39760441e-01, -2.19880220e-01, ...,
         -6.03100033e+00, -6.87910975e+00, -7.75863063e+00]],

       ...,

       [[ 1.35143729e+00,  4.38851883e-01, -4.60139628e-01, ...,
         -4.99489238e+00, -6.08962463e+00, -7.17404833e+00],
        [ 1.34832235e+00,  4.41551892e-01, -4.56055676e-01, ...,
         -5.02235170e+00, -6.11756

In [56]:
train_data.shape[1]

2

In [51]:
np.array([]).size

0

AttributeError: ignored

In [58]:
train_data.shape

(5962, 2, 10)

In [0]:
    predx= modelx.predict(inputt)

In [0]:
np.save('/content/gdrive/My Drive/indexes', testSet.indexes)

In [0]:
np.save('/content/gdrive/My Drive/data_predicted1', data_predicted)